<a href="https://colab.research.google.com/github/hsaripalli/Pump-It-Up/blob/main/model/Pump_it_up_Optuna_Tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notes**

**Hyperparamter tuned using optuna library. Best accuracy of 82.4% on submission on submission.** 

Load Data:

- Loaded and combined train and test csv files
- Dropped columns that obviously did not have any significance (mostly 0s or same value all across)
- Parsed date and created two new columns- month and year

Numeric Columns:

- List all numerical columns
- Impute gps height, lat/long using grouped means
- Impute construction year and population using grouped mean
- Created new column, age, using year recorded - construction year. Imputed negative vlaues of age
- Created new column, 'season' using the month column. 
- Using DBScan to create clusters for lat/long. Didn't do anything for accuracy

Categorical Columns

- Converted all strings to lower case 
- Split into columns that have too many unique values vs not too many unique values
- Replaced 0s and 'none's with most frequent values
- Cleaned up some values that are mostly similar but have typos or entered as different versions. For example: community vs commu. 
- Dropped some columns that are mostly similar to others

Split Train and Test

- Seperated train and test csv files after cleaning
- Did not do a train-test split to maximize the training data. Used 3 fold cross validation instead. 
- Label encoded

Pipeline:

- MyCategoryCoalescer- Customer transformer (Uncle Steve's) to retain top 25 per column and replace the rest as "Other'
- Ordinal Encoder for all category columns
- Scaler for numeric columns. Scaler didnt really boost accuracy, IMO. 

Models: 

- Trained random forest, xgboost, adaboost, bagging (with base as decision trees), extra trees, LIghGBM, CatBoost
- All models have mostly similar accuracies except adaboost. adaboost lower by a few points
- Stacking all five models gave the best accuracy

# **Load Data**

In [ ]:
# Merged train and test for preprocessing

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
values = pd.read_csv("https://raw.githubusercontent.com/hsaripalli/Pump-It-Up/main/Training_set_values.csv")
labels = pd.read_csv("https://raw.githubusercontent.com/hsaripalli/Pump-It-Up/main/Training_set_labels.csv")
test = pd.read_csv("https://raw.githubusercontent.com/hsaripalli/Pump-It-Up/main/Test_set_values.csv")

In [ ]:
# Merge train and test

values['train'] = True
test['test'] = True
data = pd.concat([values, test], ignore_index = True)

In [ ]:
#Drop columns

columns_to_drop = ['num_private', 'recorded_by']
data = data.drop(columns_to_drop, axis = 1)

In [ ]:
#Parse dates

data['date_recorded' + '_year'] = pd.to_datetime(data['date_recorded']).dt.year 
data['date_recorded' + '_month'] = pd.to_datetime(data['date_recorded']).dt.month
data = data.drop('date_recorded', axis = 1)

# **Data Cleaning - Numerical Features**

In [ ]:
numeric_columns = data.select_dtypes(exclude = 'object').columns.tolist()

In [ ]:
# Impute small latitude values with 0
data.loc[data['latitude'] > -0.5, 'latitude'] = 0

In [ ]:
# gps height and longitude: impute 0 and nan with grouped mean
col1 = ['gps_height', 'longitude', 'latitude']
data[col1] = data[col1].replace(0, np.nan)
for i in col1:
    data[i] = data[i].fillna(data.groupby('subvillage')[i].transform('mean'))
    data[i] = data[i].fillna(data.groupby('ward')[i].transform('mean'))
    data[i] = data[i].fillna(data.groupby('lga')[i].transform('mean'))
    data[i] = data[i].fillna(data.groupby('region')[i].transform('mean'))
    data[i] = data[i].fillna(data.groupby('basin')[i].transform('mean'))

In [ ]:
# construction year and population: impute 0 and nan with most frequent
col2 = ['construction_year', 'population']
data[col2] = data[col2].replace(0, np.nan)
for i in col2:
    data[i] = round(data[i].fillna(data.groupby('subvillage')[i].transform('mean')))
    data[i] = round(data[i].fillna(data.groupby('ward')[i].transform('mean')))
    data[i] = round(data[i].fillna(data.groupby('lga')[i].transform('mean')))
    data[i] = round(data[i].fillna(data.groupby('region')[i].transform('mean')))
    data[i] = round(data[i].fillna(data.groupby('basin')[i].transform('mean')))

In [ ]:
# Add age = date recordced - construction year
# Impute negative age with 1
data['age'] = data['date_recorded_year'] - data['construction_year']
data.loc[data['age'] < 0, 'age'] = 1

In [ ]:
# Jan and Feb short dry season
# long rains lasts during about March, April and May 
# long dry season lasts throughout June, July, August, September and October 
# During November and December there's another rainy season: the 'short rains'

data.loc[(data['date_recorded_month'] >= 1) & (data['date_recorded_month'] <= 2), 'season'] = 1
data.loc[(data['date_recorded_month'] >= 3) & (data['date_recorded_month'] <= 5), 'season'] = 2
data.loc[(data['date_recorded_month'] >= 6) & (data['date_recorded_month'] <= 10), 'season'] = 3
data.loc[(data['date_recorded_month'] >= 11) & (data['date_recorded_month'] <= 12), 'season'] = 4

data['season']

0        2.0
1        2.0
2        1.0
3        1.0
4        3.0
        ... 
74245    1.0
74246    2.0
74247    2.0
74248    1.0
74249    1.0
Name: season, Length: 74250, dtype: float64

In [ ]:
from sklearn.cluster import KMeans

clusters = 15
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(data[['latitude', 'longitude']].values)
kmean_feats = pd.DataFrame(kmeans.fit_transform(data[['latitude', 'longitude']].values), columns=['gspatial_' + str(i) for i in range(clusters)])


In [ ]:
data = pd.concat([data, kmean_feats], axis = 1)

# **Data Cleaning - Categorical Features**

In [ ]:
categorical_columns = data.select_dtypes(include = 'object').columns.tolist()

*Dealing with columns that contain too many unique values*




In [ ]:
# TOO MANY UNIQUE VALUES
#funder                    2140
#installer                 2410
#wpt_name                 45684
#subvillage               21425
#lga                        125
#ward                      2098
#scheme name

In [ ]:
# convert to lowercase
col3 = ['funder', 'installer','wpt_name', 'basin', 'subvillage', 'region',
                 'lga', 'ward','scheme_management', 'extraction_type','extraction_type_group',
                 'extraction_type_class','management','management_group','payment','payment_type',
                 'water_quality', 'quality_group','quantity','quantity_group','source','source_type', 
                 'source_class','waterpoint_type','waterpoint_type_group', 'scheme_name']
for i in col3:
  data[i] = data[i].str.lower()

In [ ]:
# fill na with most frequest
col4 = ['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward', 'scheme_name']
data[col4] = data[col4].replace(to_replace = ('0', 'none'), value = np.nan)

In [ ]:
data['installer'] = data['installer'].replace(to_replace = ('gover'), value = 'government')
data['installer'] = data['installer'].replace(to_replace = ('commu'), value = 'community')

In [ ]:
for i in col4:
    data[i] = data[i].fillna(data[i].mode()[0])

Dealing with columns containing **not** too many unique values




In [ ]:
# Not too many unique values
#basin                        9
#region                      21
#region_code                 27
#district_code               20
#public_meeting               2
#scheme_management           12
#permit                       2
#construction_year           55
#extraction_type             18
#extraction_type_group       13
#extraction_type_class        7
#management                  12
#management_group             5
#payment                      7
#payment_type                 7
#water_quality                8
#quality_group                6
#quantity                     5
#quantity_group               5
#source                      10
#source_type                  7
#source_class                 3
#waterpoint_type              7
#waterpoint_type_group        6
#train                        1
#test                         1
#date_recorded_year           6
#date_recorded_month         12

In [ ]:
#public_meeting               2
#scheme_management           12
#permit                       2

In [ ]:
col5 = ['public_meeting', 'permit']
for i in col5:
    data[i] = data[i].fillna(data[i].mode()[0])
    data[i] = data[i].astype(str)

In [ ]:
# public meeting and scheme management: fill na with most frequest

data['scheme_management'] = data['scheme_management'].replace(to_replace = (np.nan, 'none'), value = 'other')

In [ ]:
#extraction_type             18
#extraction_type_group       13
#extraction_type_class        7

In [ ]:
# clean/ replace some values in extraction_type column

data = data.replace({'extraction_type': 
                     {'cemo': 'other motorpump',
                      'climax': 'other motorpump',
                      'india mark ii': 'india mark',
                      'india mark iii': 'india mark',
                      'other - mkulima/shinyanga': 'other handpump',
                      'other - play pump': 'other handpump',
                      'other - rope pump': 'rope pump',
                      'other - swn 81': 'swn',
                      'swn 80': 'swn'
                      }})


In [ ]:
# describe columns (run one at a time)

#data[['extraction_type', 'extraction_type_group', 'extraction_type_class']].groupby('extraction_type_group').describe()
#data[['payment', 'payment_type']].groupby('payment').describe()
#data[['water_quality', 'quality_group']].groupby('water_quality').describe()
#data[['quantity', 'quantity_group']].groupby('quantity').describe()
#data[['source', 'source_type', 'source_class']].groupby('source').describe()
#data[['waterpoint_type', 'waterpoint_type_group']].groupby('waterpoint_type').describe()

In [ ]:
col6 = ['extraction_type_group', 'payment_type', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group']
data = data.drop(col6, axis = 1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 52 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     74250 non-null  int64  
 1   amount_tsh             74250 non-null  float64
 2   funder                 74250 non-null  object 
 3   gps_height             74250 non-null  float64
 4   installer              74250 non-null  object 
 5   longitude              74250 non-null  float64
 6   latitude               74250 non-null  float64
 7   wpt_name               74250 non-null  object 
 8   basin                  74250 non-null  object 
 9   subvillage             74250 non-null  object 
 10  region                 74250 non-null  object 
 11  region_code            74250 non-null  int64  
 12  district_code          74250 non-null  int64  
 13  lga                    74250 non-null  object 
 14  ward                   74250 non-null  object 
 15  po

# **Split Train and Test**

In [ ]:
# Reverse split merged and clean data into train and test
train_values = data[data['train'] == True]
test = data[data['test'] == True]
train_values = train_values.drop(['train', 'test'], axis = 1)
test = test.drop(['train', 'test'], axis = 1)

In [ ]:
test_set = test.drop('id', axis = 1)
x = train_values.drop('id', axis = 1)

In [ ]:
X = x.copy()
y = pd.DataFrame(labels['status_group'])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y.values.ravel())

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'functional': 0, 'functional needs repair': 1, 'non functional': 2}


In [ ]:
# Split train and test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.01, random_state = 123)

In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14850 entries, 36801 to 56271
Data columns (total 49 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             14850 non-null  float64
 1   funder                 14850 non-null  object 
 2   gps_height             14850 non-null  float64
 3   installer              14850 non-null  object 
 4   longitude              14850 non-null  float64
 5   latitude               14850 non-null  float64
 6   wpt_name               14850 non-null  object 
 7   basin                  14850 non-null  object 
 8   subvillage             14850 non-null  object 
 9   region                 14850 non-null  object 
 10  region_code            14850 non-null  int64  
 11  district_code          14850 non-null  int64  
 12  lga                    14850 non-null  object 
 13  ward                   14850 non-null  object 
 14  population             14850 non-null  float64
 15

# **Pipeline**

In [ ]:
#Uncle Steve's Custom Transformer for Category Coalescing

from sklearn.base import BaseEstimator, TransformerMixin

class MyCategoryCoalescer(BaseEstimator, TransformerMixin):
    # Coalesces (smushes/condenses) rare levels of a categorical 
    # feature into "__OTHER__".
    #
    # Will leave the `keep_top` most frequent levels unchanged; the rest
    # will be changed to `"__OTHER__"`.
    #
    # Note that there was a design choice: either have the user
    # pass in the names of the columns to operate one (which I've done here), 
    # or just operate on all the columns (and have the user be responsible for
    # passing in a subset of the dataframe). Pros and cons to each and there's
    # note a singe best answer.
    
    def __init__(self, cat_cols=[], keep_top=25):
        self.cat_cols = cat_cols
        self.keep_top = keep_top
        
        # For each cat_col, this dict will hold an list of the most-frequent 
        # levels
        self.top_n_values = {}
            
    def get_top_n_values(self, X, col, n=25):
        # A helper function to do the actual work.

        # Get the sorted value counts for the column
        vc = X[col].value_counts(sort=True, ascending=False)

        # Get the actual values
        vals = list(vc.index)
        if len(vals) > n:
            top_values = vals[0:n]
        else:
            top_values =  vals

        # Debug printing.
        #print("Top n={} values for column={}:".format(n, col))
        #print(top_values)
        return top_values
    
    def fit(self, X, y=None):

        # Find the top n values for each cateogircal column
        for col in self.cat_cols:
            self.top_n_values[col] = self.get_top_n_values(X, col, n=self.keep_top)
        return self
    
    def transform(self, X, y=None):
        _X = X.copy()
        _X[self.cat_cols] = _X[self.cat_cols].astype('category')
        for c in self.cat_cols:
            _X[c] = _X[c].cat.add_categories('__OTHER__')
            _X.loc[~_X[c].isin(self.top_n_values[c]), c] = "__OTHER__"
        return _X

In [ ]:
# Model fit

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score

categorical_cols = ['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'scheme_management',
                    'extraction_type','extraction_type_class', 'management', 'management_group', 'payment', 'water_quality',
                    'quantity','source','source_class','waterpoint_type', 'permit', 'public_meeting', 'scheme_name']

columns_to_coal = ['funder','installer', 'subvillage', 'lga', 'ward', 'wpt_name', 'scheme_name']

columns_to_scale = ['population', 'gps_height', 'latitude', 'longitude']

coalescer = MyCategoryCoalescer(cat_cols=columns_to_coal, keep_top=25)
encoder = OrdinalEncoder()
scaler = StandardScaler()


cat_transformer = Pipeline([
                            ('coalescer', coalescer),                      
                            ('encoder', encoder)
                           ])

preprocessor = Pipeline(steps = [
                                 ('ct', ColumnTransformer(
                                     transformers=[
                                                   ('categorical', cat_transformer, categorical_cols),
                                                   ('scale', scaler, columns_to_scale)
                                                   ], 
                                                   remainder = 'passthrough', 
                                                   sparse_threshold =0)),
                                 ])

# *Random Forest (optuna tuned)*

In [ ]:
# Random Forest Tuned
# Random Forest Tuned Hyper Parameters
# {'rf__max_depth': 20, 'rf__min_samples_split': 5, 'rf__n_estimators': 1000}

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion = 'gini',
                            n_estimators = 536,
                            min_samples_split = 8,
                            max_depth = 20,
                            random_state = 42)

rf_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('rf', rf)])
rf_pipeline.fit(X_train,y_train)
y_pred_rf_pipeline = rf_pipeline.predict(X_test)

In [ ]:
print("Accuracy of RF = {:.4f}".format(accuracy_score(y_test, y_pred_rf_pipeline)))

Accuracy of RF = 0.8418


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_rf_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_rf_pipeline))

In [ ]:
import optuna
def objective(trial):
    

    param = {
        "criterion": trial.suggest_categorical("criterion", ['gini', 'entropy']),
        "min_samples_split": trial.suggest_int("min_samples_split", 2,10),
        "n_estimators": trial.suggest_int("n_estimators", 200,1500),
        "max_depth": trial.suggest_int("max_depth", 5, 50)
    }

    rf = RandomForestClassifier(**param)
    rf_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('rf', rf)])
    
    return cross_val_score(rf_pipeline, X, y, cv = 3).mean()
    
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-11-16 10:40:02,033] A new study created in memory with name: no-name-4e962812-b08c-43ac-916e-97406778cbe3
[I 2021-11-16 10:42:50,490] Trial 0 finished with value: 0.7937205387205388 and parameters: {'criterion': 'gini', 'min_samples_split': 10, 'n_estimators': 470, 'max_depth': 14}. Best is trial 0 with value: 0.7937205387205388.
[I 2021-11-16 11:01:18,854] Trial 1 finished with value: 0.806969696969697 and parameters: {'criterion': 'entropy', 'min_samples_split': 4, 'n_estimators': 1430, 'max_depth': 49}. Best is trial 1 with value: 0.806969696969697.
[I 2021-11-16 11:07:59,682] Trial 2 finished with value: 0.8093771043771044 and parameters: {'criterion': 'entropy', 'min_samples_split': 7, 'n_estimators': 516, 'max_depth': 26}. Best is trial 2 with value: 0.8093771043771044.
[I 2021-11-16 11:13:46,526] Trial 3 finished with value: 0.8093602693602694 and parameters: {'criterion': 'gini', 'min_samples_split': 9, 'n_estimators': 580, 'max_depth': 46}. Best is trial 2 with value: 

Number of finished trials: 10
Best trial:
  Value: 0.8093771043771044
  Params: 
    criterion: entropy
    min_samples_split: 7
    n_estimators: 516
    max_depth: 26


In [ ]:
#Number of finished trials: 10
#Best trial:
#  Value: 0.8093771043771044
#  Params: 
#    criterion: entropy
#    min_samples_split: 7
#    n_estimators: 516
#    max_depth: 26
        
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion = 'gini',
                            n_estimators = 516,
                            min_samples_split = 7,
                            max_depth = 26,
                            random_state = 42)

rf_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('rf', rf)])
rf_pipeline.fit(X_train,y_train)
y_pred_rf_pipeline = rf_pipeline.predict(X_test)

In [ ]:
print("Accuracy of RF = {:.4f}".format(accuracy_score(y_test, y_pred_rf_pipeline)))

Accuracy of RF = 0.8468


# *LGBM (optuna tuned)*

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(boosting_type = 'gbdt',
                      objective = 'multiclass',
                      num_class = 3,
                      metric = 'multi_error',
                      num_iterations = 200,
                      lambda_l1 =  2.2899315163770417e-06,
                      lambda_l2 =  2.6273452242794607e-06,
                      num_leaves = 239,
                      feature_fraction = 0.5633644014015632,
                      learning_rate = 0.06012805964180289,
                      bagging_fraction = 0.6953776886469089,
                      bagging_freq = 6,
                      min_child_samples = 47,
                      min_data_in_leaf = 17,
                      max_depth = 46
                      )

lgbm_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('lgbm', lgbm)])
lgbm_pipeline.fit(X_train,y_train)
y_pred_lgbm_pipeline = lgbm_pipeline.predict(X_test)

C:\Users\HariSaripalli\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.5633644014015632, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5633644014015632
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] lambda_l1 is set=2.2899315163770417e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.2899315163770417e-06
[LightGBM] [Warning] bagging_fraction is set=0.6953776886469089, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6953776886469089
[LightGBM] [Warning] lambda_l2 is set=2.6273452242794607e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.6273452242794607e-06
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


In [ ]:
print("Accuracy of LGBM   = {:.4f}".format(accuracy_score(y_test, y_pred_lgbm_pipeline)))

Accuracy of LGBM   = 0.8333


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_lgbm_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_lgbm_pipeline))

# *Catboost (hyperparameter tuned)*

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(depth = 10,
                        iterations = 500,
                         learning_rate = 0.05,
                        random_state = 42)

cat_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('catboost', cat)])
cat_pipeline.fit(X_train,y_train)
y_pred_cat_pipeline = cat_pipeline.predict(X_test)

In [ ]:
print("Accuracy of Catboost   = {:.4f}".format(accuracy_score(y_test, y_pred_cat_pipeline)))

In [ ]:
import optuna
from catboost import CatBoostClassifier

def objective(trial):
    

    param = {
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        
        "used_ram_limit": "2gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    cat = CatBoostClassifier(**param)
    cat_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('catboost', cat)])
    
    return cross_val_score(cat_pipeline, X, y, cv = 3).mean()
    
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
#Number of finished trials: 10
#Best trial:
#  Value: 0.8049831649831649
#  Params: 
#    colsample_bylevel: 0.07369920952387737
#    depth: 12
#    boosting_type: Plain
#   bootstrap_type: MVS

from catboost import CatBoostClassifier

cat = CatBoostClassifier(colsample_bylevel = 0.073699209523,
                         depth = 12,
                         boosting_type = 'Plain',
                         bootstrap_type = 'MVS',
                        random_state = 42)

cat_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('catboost', cat)])
cat_pipeline.fit(X_train,y_train)
y_pred_cat_pipeline = cat_pipeline.predict(X_test)

In [ ]:
print("Accuracy of Catboost   = {:.4f}".format(accuracy_score(y_test, y_pred_cat_pipeline)))

Accuracy of Catboost   = 0.8316


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_cat_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_cat_pipeline))

# *XG Boost*

In [ ]:
from xgboost import XGBClassifier

#Number of finished trials: 30
#Best trial:
#  Value: 0.8013131313131314
#  Params: 
#    booster: dart
#    lambda: 4.572637572518502e-07
#    alpha: 6.037662427475617e-05
#    subsample: 0.7162353406216146
#    colsample_bytree: 0.8486248682584188
#    max_depth: 7
#    min_child_weight: 9
#    eta: 0.3563123559925298
#    gamma: 5.017895421049517e-05
#    grow_policy: depthwise
#    sample_type: uniform
#    normalize_type: forest
#    rate_drop: 0.012104590680294654
#    skip_drop: 0.00036189755567904127

xg = XGBClassifier(booster = 'dart',
                   alpha =  6.037662427475617e-05,
                   subsample = 0.7162353406216146,
                   colsample_bytree = 0.8486248682584188,
                   max_depth = 7,
                   min_child_weight = 9,
                   eta = 0.3563123559925298,
                   gamma = 5.017895421049517e-05,
                   grow_policy = 'depthwise',
                   sample_type = 'uniform',
                   normalize_type = 'forest',
                   rate_drop = 0.012104590680294654,
                   skip_drop = 0.00036189755567904127,
                   objective='multi:softmax',
                   use_label_encoder = False)

xg_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('xgboost', xg)])
xg_pipeline.fit(X_train,y_train)
y_pred_xg_pipeline = xg_pipeline.predict(X_test)

In [ ]:
print("Accuracy of XGB   = {:.4f}".format(accuracy_score(y_test, y_pred_xg_pipeline)))

In [ ]:
import optuna

def objective(trial):
    
  
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    xg = XGBClassifier(**param)
    xg_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('xgboost', xg)])
    
    return cross_val_score(xg_pipeline, X, y, cv = 3).mean()


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_xg_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_xg_pipeline))

# *Extra Trees*

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

xt = ExtraTreesClassifier(n_estimators=200,
                          random_state=42)

xt_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('extra trees', xt)])
xt_pipeline.fit(X_train,y_train)
y_pred_xt_pipeline = xt_pipeline.predict(X_test)

In [ ]:
print("Accuracy of EXTRA TREES   = {:.4f}".format(accuracy_score(y_test, y_pred_xt_pipeline)))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_xt_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_xt_pipeline))

# *Bagging*

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

bag =  BaggingClassifier(n_estimators=100,
                         max_features = 0.5,
                         random_state=42)

bag_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('bagging', bag)])
bag_pipeline.fit(X_train,y_train)
y_pred_bag_pipeline = bag_pipeline.predict(X_test)

In [ ]:
print("Accuracy of BAGGING   = {:.4f}".format(accuracy_score(y_test, y_pred_bag_pipeline)))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_bag_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_bag_pipeline))

# *Voting Classifier*

In [ ]:
from sklearn.ensemble import VotingClassifier

est_list = [('rf', rf), ('xgboost', xg), ('lgbm', lgbm)]


vclf = VotingClassifier(estimators = est_list, voting='soft')


vote_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('voting', vclf)])

vote_pipeline.fit(X_train,y_train)
y_pred_vote_pipeline = vote_pipeline.predict(X_test)

In [ ]:
print("Accuracy of VOTING = {:.4f}".format(accuracy_score(y_test, y_pred_vote_pipeline)))

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


rf = RandomForestClassifier(criterion = 'gini',
                            n_estimators = 536,
                            min_samples_split = 8,
                            max_depth = 20,
                            random_state = 42)

xg = XGBClassifier(booster = 'dart',
                   alpha =  6.037662427475617e-05,
                   subsample = 0.7162353406216146,
                   colsample_bytree = 0.8486248682584188,
                   max_depth = 7,
                   min_child_weight = 9,
                   eta = 0.3563123559925298,
                   gamma = 5.017895421049517e-05,
                   grow_policy = 'depthwise',
                   sample_type = 'uniform',
                   normalize_type = 'forest',
                   rate_drop = 0.012104590680294654,
                   skip_drop = 0.00036189755567904127,
                   objective='multi:softmax',
                   use_label_encoder = False)

xt = ExtraTreesClassifier(n_estimators=200,
                          random_state=42)


bag =  BaggingClassifier(n_estimators=100,
                         max_features = 0.5,
                         random_state=42)

cat = CatBoostClassifier(colsample_bylevel = 0.073699209523,
                         depth = 12,
                         boosting_type = 'Plain',
                         bootstrap_type = 'MVS',
                        random_state = 42)

lgbm = LGBMClassifier(boosting_type = 'gbdt',
                      objective = 'multiclass',
                      num_class = 3,
                      metric = 'multi_error',
                      num_iterations = 200,
                      lambda_l1 =  2.2899315163770417e-06,
                      lambda_l2 =  2.6273452242794607e-06,
                      num_leaves = 239,
                      feature_fraction = 0.5633644014015632,
                      learning_rate = 0.06012805964180289,
                      bagging_fraction = 0.6953776886469089,
                      bagging_freq = 6,
                      min_child_samples = 47,
                      min_data_in_leaf = 17,
                      max_depth = 46
                      )

rf_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('rf', rf)])
xg_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('xgboost', xg)])
xt_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('extra trees', xt)])
bag_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('bagging', bag)])
cat_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('catboost', cat)])
lgbm_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('lgbm', lgbm)])


est_list = [('rf', rf), ('xgboost', xg), ('extra trees', xt), ('bagging', bag), ('catboost', cat), ('lgbm', lgbm)]
vclf = VotingClassifier(estimators = est_list, voting='soft')


vote_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('voting', vclf)])

vote_pipeline.fit(X,y)

[13:08:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.097451
0:	learn: 1.0659227	total: 8.11ms	remaining: 8.1s
1:	learn: 1.0031807	total: 165ms	remaining: 1m 22s
2:	learn: 0.9601727	total: 339ms	remaining: 1m 52s
3:	learn: 0.9240684	total: 562ms	remaining: 2m 19s
4:	learn: 0.9073494	total: 569ms	remaining: 1m 53s
5:	learn: 0.8777925	total: 582ms	remaining: 1m 36s
6:	learn: 0.8447936	total: 719ms	remaining: 1m 41s
7:	learn: 0.8210596	total: 735ms	remaining: 1m 31s
8:	learn: 0.7980529	total: 812ms	remaining: 1m 29s
9:	learn: 0.7747224	total: 1.04s	remaining: 1m 43s
10:	learn: 0.7552244	total: 1.28s	remaining: 1m 55s
11:	learn: 0.7403637	total: 1.46s	remaining: 1m 59s
12:	learn: 0.7238737	total: 1.59s	remaining: 2

152:	learn: 0.4403373	total: 18.8s	remaining: 1m 44s
153:	learn: 0.4402025	total: 18.8s	remaining: 1m 43s
154:	learn: 0.4393434	total: 19s	remaining: 1m 43s
155:	learn: 0.4393119	total: 19s	remaining: 1m 42s
156:	learn: 0.4388860	total: 19.2s	remaining: 1m 42s
157:	learn: 0.4387282	total: 19.2s	remaining: 1m 42s
158:	learn: 0.4380548	total: 19.4s	remaining: 1m 42s
159:	learn: 0.4374224	total: 19.5s	remaining: 1m 42s
160:	learn: 0.4366628	total: 19.7s	remaining: 1m 42s
161:	learn: 0.4358775	total: 19.9s	remaining: 1m 42s
162:	learn: 0.4352856	total: 20.1s	remaining: 1m 43s
163:	learn: 0.4346384	total: 20.2s	remaining: 1m 43s
164:	learn: 0.4341636	total: 20.4s	remaining: 1m 43s
165:	learn: 0.4331841	total: 20.5s	remaining: 1m 43s
166:	learn: 0.4324974	total: 20.7s	remaining: 1m 43s
167:	learn: 0.4324662	total: 20.7s	remaining: 1m 42s
168:	learn: 0.4318275	total: 20.8s	remaining: 1m 42s
169:	learn: 0.4317280	total: 20.9s	remaining: 1m 41s
170:	learn: 0.4311413	total: 21s	remaining: 1m 41s

309:	learn: 0.3783245	total: 39.1s	remaining: 1m 27s
310:	learn: 0.3779677	total: 39.3s	remaining: 1m 26s
311:	learn: 0.3776412	total: 39.4s	remaining: 1m 26s
312:	learn: 0.3774973	total: 39.4s	remaining: 1m 26s
313:	learn: 0.3770581	total: 39.6s	remaining: 1m 26s
314:	learn: 0.3766438	total: 39.8s	remaining: 1m 26s
315:	learn: 0.3766273	total: 39.8s	remaining: 1m 26s
316:	learn: 0.3761340	total: 40s	remaining: 1m 26s
317:	learn: 0.3761145	total: 40s	remaining: 1m 25s
318:	learn: 0.3757222	total: 40.2s	remaining: 1m 25s
319:	learn: 0.3754326	total: 40.3s	remaining: 1m 25s
320:	learn: 0.3749786	total: 40.5s	remaining: 1m 25s
321:	learn: 0.3747553	total: 40.6s	remaining: 1m 25s
322:	learn: 0.3744022	total: 40.8s	remaining: 1m 25s
323:	learn: 0.3738675	total: 40.9s	remaining: 1m 25s
324:	learn: 0.3736812	total: 41.2s	remaining: 1m 25s
325:	learn: 0.3732727	total: 41.3s	remaining: 1m 25s
326:	learn: 0.3731170	total: 41.5s	remaining: 1m 25s
327:	learn: 0.3728004	total: 41.6s	remaining: 1m 2

465:	learn: 0.3403127	total: 58.6s	remaining: 1m 7s
466:	learn: 0.3399554	total: 58.8s	remaining: 1m 7s
467:	learn: 0.3395683	total: 58.9s	remaining: 1m 7s
468:	learn: 0.3394495	total: 59.1s	remaining: 1m 6s
469:	learn: 0.3394357	total: 59.1s	remaining: 1m 6s
470:	learn: 0.3391082	total: 59.3s	remaining: 1m 6s
471:	learn: 0.3390259	total: 59.3s	remaining: 1m 6s
472:	learn: 0.3390235	total: 59.3s	remaining: 1m 6s
473:	learn: 0.3390204	total: 59.3s	remaining: 1m 5s
474:	learn: 0.3389819	total: 59.3s	remaining: 1m 5s
475:	learn: 0.3388426	total: 59.5s	remaining: 1m 5s
476:	learn: 0.3386892	total: 59.7s	remaining: 1m 5s
477:	learn: 0.3386892	total: 59.7s	remaining: 1m 5s
478:	learn: 0.3384678	total: 59.9s	remaining: 1m 5s
479:	learn: 0.3382929	total: 1m	remaining: 1m 5s
480:	learn: 0.3380034	total: 1m	remaining: 1m 4s
481:	learn: 0.3378478	total: 1m	remaining: 1m 4s
482:	learn: 0.3375705	total: 1m	remaining: 1m 4s
483:	learn: 0.3374383	total: 1m	remaining: 1m 4s
484:	learn: 0.3372454	total

624:	learn: 0.3130067	total: 1m 20s	remaining: 48.4s
625:	learn: 0.3130060	total: 1m 20s	remaining: 48.2s
626:	learn: 0.3128506	total: 1m 20s	remaining: 48.1s
627:	learn: 0.3126651	total: 1m 21s	remaining: 48.1s
628:	learn: 0.3126646	total: 1m 21s	remaining: 47.9s
629:	learn: 0.3124870	total: 1m 21s	remaining: 47.7s
630:	learn: 0.3122439	total: 1m 21s	remaining: 47.6s
631:	learn: 0.3120500	total: 1m 21s	remaining: 47.5s
632:	learn: 0.3118759	total: 1m 21s	remaining: 47.4s
633:	learn: 0.3117841	total: 1m 21s	remaining: 47.2s
634:	learn: 0.3117537	total: 1m 21s	remaining: 47s
635:	learn: 0.3116105	total: 1m 21s	remaining: 46.9s
636:	learn: 0.3114653	total: 1m 22s	remaining: 46.7s
637:	learn: 0.3111677	total: 1m 22s	remaining: 46.7s
638:	learn: 0.3109383	total: 1m 22s	remaining: 46.6s
639:	learn: 0.3107524	total: 1m 22s	remaining: 46.4s
640:	learn: 0.3105525	total: 1m 22s	remaining: 46.4s
641:	learn: 0.3104549	total: 1m 22s	remaining: 46.2s
642:	learn: 0.3102564	total: 1m 22s	remaining: 4

781:	learn: 0.2896919	total: 1m 42s	remaining: 28.4s
782:	learn: 0.2895107	total: 1m 42s	remaining: 28.3s
783:	learn: 0.2894123	total: 1m 42s	remaining: 28.2s
784:	learn: 0.2894107	total: 1m 42s	remaining: 28s
785:	learn: 0.2892354	total: 1m 42s	remaining: 27.9s
786:	learn: 0.2892319	total: 1m 42s	remaining: 27.8s
787:	learn: 0.2891491	total: 1m 42s	remaining: 27.6s
788:	learn: 0.2890310	total: 1m 42s	remaining: 27.5s
789:	learn: 0.2888265	total: 1m 43s	remaining: 27.4s
790:	learn: 0.2886883	total: 1m 43s	remaining: 27.3s
791:	learn: 0.2884643	total: 1m 43s	remaining: 27.2s
792:	learn: 0.2884642	total: 1m 43s	remaining: 27s
793:	learn: 0.2883599	total: 1m 43s	remaining: 26.9s
794:	learn: 0.2883599	total: 1m 43s	remaining: 26.7s
795:	learn: 0.2882717	total: 1m 43s	remaining: 26.6s
796:	learn: 0.2881264	total: 1m 44s	remaining: 26.5s
797:	learn: 0.2880070	total: 1m 44s	remaining: 26.4s
798:	learn: 0.2877037	total: 1m 44s	remaining: 26.2s
799:	learn: 0.2876089	total: 1m 44s	remaining: 26.

938:	learn: 0.2709692	total: 2m 4s	remaining: 8.07s
939:	learn: 0.2707305	total: 2m 4s	remaining: 7.94s
940:	learn: 0.2705530	total: 2m 4s	remaining: 7.81s
941:	learn: 0.2704053	total: 2m 4s	remaining: 7.68s
942:	learn: 0.2702779	total: 2m 4s	remaining: 7.55s
943:	learn: 0.2701672	total: 2m 5s	remaining: 7.42s
944:	learn: 0.2700988	total: 2m 5s	remaining: 7.29s
945:	learn: 0.2700723	total: 2m 5s	remaining: 7.16s
946:	learn: 0.2699815	total: 2m 5s	remaining: 7.03s
947:	learn: 0.2699726	total: 2m 5s	remaining: 6.89s
948:	learn: 0.2698101	total: 2m 5s	remaining: 6.76s
949:	learn: 0.2697199	total: 2m 5s	remaining: 6.63s
950:	learn: 0.2696275	total: 2m 6s	remaining: 6.5s
951:	learn: 0.2695372	total: 2m 6s	remaining: 6.37s
952:	learn: 0.2695372	total: 2m 6s	remaining: 6.23s
953:	learn: 0.2694499	total: 2m 6s	remaining: 6.1s
954:	learn: 0.2694319	total: 2m 6s	remaining: 5.96s
955:	learn: 0.2692543	total: 2m 6s	remaining: 5.83s
956:	learn: 0.2691688	total: 2m 6s	remaining: 5.7s
957:	learn: 0.2

C:\Users\HariSaripalli\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.5633644014015632, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5633644014015632
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] lambda_l1 is set=2.2899315163770417e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.2899315163770417e-06
[LightGBM] [Warning] bagging_fraction is set=0.6953776886469089, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6953776886469089
[LightGBM] [Warning] lambda_l2 is set=2.6273452242794607e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.6273452242794607e-06
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


Pipeline(steps=[('preprocess',
                 Pipeline(steps=[('ct',
                                  ColumnTransformer(remainder='passthrough',
                                                    sparse_threshold=0,
                                                    transformers=[('categorical',
                                                                   Pipeline(steps=[('coalescer',
                                                                                    MyCategoryCoalescer(cat_cols=['funder',
                                                                                                                  'installer',
                                                                                                                  'subvillage',
                                                                                                                  'lga',
                                                                                                     

In [ ]:
accuracy = cross_val_score(vote_pipeline, X, y, cv = 5).mean()
accuracy

[15:06:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.096348
0:	learn: 1.0662106	total: 25.5ms	remaining: 25.4s
1:	learn: 1.0055175	total: 208ms	remaining: 1m 43s
2:	learn: 0.9641282	total: 386ms	remaining: 2m 8s
3:	learn: 0.9284220	total: 604ms	remaining: 2m 30s
4:	learn: 0.9116710	total: 608ms	remaining: 2m 1s
5:	learn: 0.8820628	total: 622ms	remaining: 1m 42s
6:	learn: 0.8492782	total: 756ms	remaining: 1m 47s
7:	learn: 0.8254330	total: 772ms	remaining: 1m 35s
8:	learn: 0.8012324	total: 947ms	remaining: 1m 44s
9:	learn: 0.7782813	total: 1.11s	remaining: 1m 50s
10:	learn: 0.7607554	total: 1.3s	remaining: 1m 56s
11:	learn: 0.7457564	total: 1.54s	remaining: 2m 7s
12:	learn: 0.7400120	total: 1.55s	remaining: 1m 5

152:	learn: 0.4375658	total: 17.2s	remaining: 1m 35s
153:	learn: 0.4374834	total: 17.2s	remaining: 1m 34s
154:	learn: 0.4366718	total: 17.4s	remaining: 1m 34s
155:	learn: 0.4366372	total: 17.4s	remaining: 1m 34s
156:	learn: 0.4357443	total: 17.6s	remaining: 1m 34s
157:	learn: 0.4355994	total: 17.6s	remaining: 1m 33s
158:	learn: 0.4350086	total: 17.7s	remaining: 1m 33s
159:	learn: 0.4343317	total: 17.9s	remaining: 1m 33s
160:	learn: 0.4336795	total: 18s	remaining: 1m 33s
161:	learn: 0.4329200	total: 18.1s	remaining: 1m 33s
162:	learn: 0.4320716	total: 18.3s	remaining: 1m 33s
163:	learn: 0.4316295	total: 18.4s	remaining: 1m 33s
164:	learn: 0.4311422	total: 18.5s	remaining: 1m 33s
165:	learn: 0.4304690	total: 18.6s	remaining: 1m 33s
166:	learn: 0.4296733	total: 18.7s	remaining: 1m 33s
167:	learn: 0.4296453	total: 18.8s	remaining: 1m 32s
168:	learn: 0.4291134	total: 18.9s	remaining: 1m 32s
169:	learn: 0.4289988	total: 18.9s	remaining: 1m 32s
170:	learn: 0.4281587	total: 19.1s	remaining: 1m

308:	learn: 0.3704911	total: 35.7s	remaining: 1m 19s
309:	learn: 0.3701856	total: 35.9s	remaining: 1m 19s
310:	learn: 0.3697434	total: 36s	remaining: 1m 19s
311:	learn: 0.3695011	total: 36.1s	remaining: 1m 19s
312:	learn: 0.3693657	total: 36.2s	remaining: 1m 19s
313:	learn: 0.3690623	total: 36.3s	remaining: 1m 19s
314:	learn: 0.3686133	total: 36.5s	remaining: 1m 19s
315:	learn: 0.3685828	total: 36.5s	remaining: 1m 18s
316:	learn: 0.3678970	total: 36.6s	remaining: 1m 18s
317:	learn: 0.3678720	total: 36.7s	remaining: 1m 18s
318:	learn: 0.3674194	total: 36.8s	remaining: 1m 18s
319:	learn: 0.3669094	total: 36.9s	remaining: 1m 18s
320:	learn: 0.3663014	total: 37.1s	remaining: 1m 18s
321:	learn: 0.3659432	total: 37.3s	remaining: 1m 18s
322:	learn: 0.3656073	total: 37.4s	remaining: 1m 18s
323:	learn: 0.3650639	total: 37.6s	remaining: 1m 18s
324:	learn: 0.3647431	total: 37.7s	remaining: 1m 18s
325:	learn: 0.3645442	total: 37.9s	remaining: 1m 18s
326:	learn: 0.3642978	total: 38s	remaining: 1m 1

465:	learn: 0.3319020	total: 54.3s	remaining: 1m 2s
466:	learn: 0.3316974	total: 54.4s	remaining: 1m 2s
467:	learn: 0.3313906	total: 54.6s	remaining: 1m 2s
468:	learn: 0.3311284	total: 54.7s	remaining: 1m 1s
469:	learn: 0.3307859	total: 54.8s	remaining: 1m 1s
470:	learn: 0.3305874	total: 54.8s	remaining: 1m 1s
471:	learn: 0.3303364	total: 55s	remaining: 1m 1s
472:	learn: 0.3302645	total: 55s	remaining: 1m 1s
473:	learn: 0.3302601	total: 55s	remaining: 1m 1s
474:	learn: 0.3302549	total: 55s	remaining: 1m
475:	learn: 0.3302168	total: 55.1s	remaining: 1m
476:	learn: 0.3300163	total: 55.2s	remaining: 1m
477:	learn: 0.3296675	total: 55.4s	remaining: 1m
478:	learn: 0.3296675	total: 55.4s	remaining: 1m
479:	learn: 0.3295009	total: 55.6s	remaining: 1m
480:	learn: 0.3292071	total: 55.7s	remaining: 1m
481:	learn: 0.3287889	total: 55.9s	remaining: 1m
482:	learn: 0.3286466	total: 56.1s	remaining: 1m
483:	learn: 0.3282715	total: 56.3s	remaining: 60s
484:	learn: 0.3280302	total: 56.4s	remaining: 59.

624:	learn: 0.3008819	total: 1m 13s	remaining: 44.2s
625:	learn: 0.3008812	total: 1m 13s	remaining: 44s
626:	learn: 0.3007533	total: 1m 13s	remaining: 43.9s
627:	learn: 0.3004857	total: 1m 13s	remaining: 43.8s
628:	learn: 0.3004853	total: 1m 13s	remaining: 43.6s
629:	learn: 0.3002665	total: 1m 14s	remaining: 43.5s
630:	learn: 0.3000554	total: 1m 14s	remaining: 43.4s
631:	learn: 0.2998818	total: 1m 14s	remaining: 43.3s
632:	learn: 0.2997265	total: 1m 14s	remaining: 43.2s
633:	learn: 0.2996635	total: 1m 14s	remaining: 43.1s
634:	learn: 0.2996199	total: 1m 14s	remaining: 42.9s
635:	learn: 0.2995086	total: 1m 14s	remaining: 42.7s
636:	learn: 0.2993318	total: 1m 14s	remaining: 42.6s
637:	learn: 0.2990376	total: 1m 14s	remaining: 42.5s
638:	learn: 0.2988515	total: 1m 15s	remaining: 42.4s
639:	learn: 0.2986224	total: 1m 15s	remaining: 42.3s
640:	learn: 0.2983358	total: 1m 15s	remaining: 42.2s
641:	learn: 0.2982207	total: 1m 15s	remaining: 42.1s
642:	learn: 0.2980776	total: 1m 15s	remaining: 4

...

766:	learn: 0.2766456	total: 1m 34s	remaining: 28.8s
767:	learn: 0.2766456	total: 1m 34s	remaining: 28.6s
768:	learn: 0.2765275	total: 1m 34s	remaining: 28.5s
769:	learn: 0.2763961	total: 1m 34s	remaining: 28.4s
770:	learn: 0.2763903	total: 1m 35s	remaining: 28.2s
771:	learn: 0.2762707	total: 1m 35s	remaining: 28.1s
772:	learn: 0.2761704	total: 1m 35s	remaining: 28s
773:	learn: 0.2760923	total: 1m 35s	remaining: 27.9s
774:	learn: 0.2760483	total: 1m 35s	remaining: 27.7s
775:	learn: 0.2759176	total: 1m 35s	remaining: 27.6s
776:	learn: 0.2758549	total: 1m 35s	remaining: 27.5s
777:	learn: 0.2757022	total: 1m 35s	remaining: 27.3s
778:	learn: 0.2755953	total: 1m 35s	remaining: 27.2s
779:	learn: 0.2754764	total: 1m 36s	remaining: 27.1s
780:	learn: 0.2752099	total: 1m 36s	remaining: 27s
781:	learn: 0.2750841	total: 1m 36s	remaining: 26.9s
782:	learn: 0.2750828	total: 1m 36s	remaining: 26.7s
783:	learn: 0.2749019	total: 1m 36s	remaining: 26.6s
784:	learn: 0.2748969	total: 1m 36s	remaining: 26.

923:	learn: 0.2587206	total: 1m 54s	remaining: 9.42s
924:	learn: 0.2586718	total: 1m 54s	remaining: 9.3s
925:	learn: 0.2585531	total: 1m 54s	remaining: 9.18s
926:	learn: 0.2583531	total: 1m 55s	remaining: 9.06s
927:	learn: 0.2582359	total: 1m 55s	remaining: 8.94s
928:	learn: 0.2581534	total: 1m 55s	remaining: 8.81s
929:	learn: 0.2580092	total: 1m 55s	remaining: 8.7s
930:	learn: 0.2579288	total: 1m 55s	remaining: 8.57s
931:	learn: 0.2578830	total: 1m 55s	remaining: 8.45s
932:	learn: 0.2578141	total: 1m 55s	remaining: 8.33s
933:	learn: 0.2577178	total: 1m 56s	remaining: 8.2s
934:	learn: 0.2576538	total: 1m 56s	remaining: 8.08s
935:	learn: 0.2575119	total: 1m 56s	remaining: 7.96s
936:	learn: 0.2573300	total: 1m 56s	remaining: 7.83s
937:	learn: 0.2572299	total: 1m 56s	remaining: 7.71s
938:	learn: 0.2570701	total: 1m 56s	remaining: 7.59s
939:	learn: 0.2569746	total: 1m 57s	remaining: 7.47s
940:	learn: 0.2568895	total: 1m 57s	remaining: 7.35s
941:	learn: 0.2568295	total: 1m 57s	remaining: 7.

C:\Users\HariSaripalli\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.5633644014015632, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5633644014015632
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] lambda_l1 is set=2.2899315163770417e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.2899315163770417e-06
[LightGBM] [Warning] bagging_fraction is set=0.6953776886469089, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6953776886469089
[LightGBM] [Warning] lambda_l2 is set=2.6273452242794607e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.6273452242794607e-06
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


0.8176430976430977

In [ ]:
accuracy

0.8176430976430977

In [ ]:
print("Accuracy of VOTING = {:.4f}".format(accuracy_score(y_test, y_pred_voting_pipeline)))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_vote_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_vote_pipeline))

# *Stacking*

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

#est_list = [('rf', rf), ('xgboost', xg), ('extra trees', xt), ('bagging', bag), ('catboost', cat), ('lgbm', lgbm)]
est_list = [('rf', rf), ('xgboost', xg), ('lgbm', lgbm)]

sclf = StackingClassifier(estimators = est_list,
                          final_estimator = LogisticRegression())

stacking_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('stacking', sclf)])

stacking_pipeline.fit(X_train,y_train)
y_pred_stacking_pipeline = stacking_pipeline.predict(X_test)

In [ ]:
print("Accuracy of STACKING = {:.4f}".format(accuracy_score(y_test, y_pred_stacking_pipeline)))

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

est_list = [('rf', rf), ('xgboost', xg), ('extra trees', xt), ('bagging', bag), ('catboost', cat), ('lgbm', lgbm)]
#est_list = [('rf', rf), ('xgboost', xg), ('lgbm', lgbm)]

sclf = StackingClassifier(estimators = est_list,
                          final_estimator = LogisticRegression())

stacking_pipeline = Pipeline(steps = [('preprocess', preprocessor), ('stacking', sclf)])

stacking_pipeline.fit(X_train,y_train)
y_pred_stacking_pipeline = stacking_pipeline.predict(X_test)

In [ ]:
print("Accuracy of STACKING = {:.4f}".format(accuracy_score(y_test, y_pred_stacking_pipeline)))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred_stacking_pipeline))
pd.DataFrame(confusion_matrix(y_test, y_pred_stacking_pipeline))

# **Predictions to CSV**

In [ ]:
# Predictions
# Uncomment whichever model's prediction is desired

#RF
#y_pred_test = rf_pipeline.predict(test_set)

#XGBoost
#y_pred_test = xg_pipeline.predict(test_set)

#Extra Trees
#y_pred_test = xt_pipeline.predict(test_set)

#Stacking
#y_pred_test = stacking_pipeline.predict(test_set)

#Voting
y_pred_test = vote_pipeline.predict(test_set)

#{'functional': 0, 'functional needs repair': 1, 'non functional': 2}

In [ ]:
predictions = pd.DataFrame(
                            {'id': test.id,
                           'status_group': y_pred_test}
                         )
predictions

,id,status_group
59400,50785,2
59401,51630,0
59402,17168,0
59403,45559,2
59404,49871,0
...,...,...
74245,39307,2
74246,18990,0
74247,28749,0
74248,33492,0


In [ ]:
predictions.loc[predictions['status_group'] == 0, 'status_group'] = 'functional'
predictions.loc[predictions['status_group'] == 1, 'status_group'] = 'functional needs repair'
predictions.loc[predictions['status_group'] == 2, 'status_group'] = 'non functional'

In [ ]:
predictions

,id,status_group
59400,50785,non functional
59401,51630,functional
59402,17168,functional
59403,45559,non functional
59404,49871,functional
...,...,...
74245,39307,non functional
74246,18990,functional
74247,28749,functional
74248,33492,functional


In [ ]:
# Saving file
predictions.to_csv('my_submission.csv', header=True, index=False)

#from google.colab import files
#files.download('my_submission.csv')